In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

<b> Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model to make detections
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), #landmark color 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)  #connection color
                             ) 

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<b> Extract Keypoint Values

In [62]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4) #no. of landmarks * params in landmark
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

<b> Folder Setup for Collection

In [9]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [64]:
#Make directories and subdirectories for each action
for action in actions: 
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

<b> Collect keypoint values for training and testing

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic for breaks between frames
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

<b> Preprocess data and create labels and features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence + 1), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
x = np.array(sequences)

In [16]:
y = to_categorical(labels).astype(int)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

Build and Train LSTM Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()    #sequentially added layers
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))   # 64 units in the layer
model.add(LSTM(128, return_sequences=True, activation='relu'))  
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))   #Dense does not take any units
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

<b>why mediapipe and lstm training model is used?
1. less data required
2. we can create a more dense neural network
3. faster to train since we took in more layers
4. NN was simpler hence making detection easier

In [22]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#optimizer can be any; loss is constant, this function used for multi class classfication model;metrics to track accuracy(optional)

In [25]:
#training

model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 804ms/step - loss: 2.9051 - categorical_accuracy: 0.3412
Epoch 2/2000
3/3 [==============================] - 0s 83ms/step - loss: 12.7229 - categorical_accuracy: 0.3294
Epoch 3/2000
3/3 [==============================] - 0s 124ms/step - loss: 10.7143 - categorical_accuracy: 0.3765
Epoch 4/2000
3/3 [==============================] - 0s 108ms/step - loss: 103.8845 - categorical_accuracy: 0.2588
Epoch 5/2000
3/3 [==============================] - 0s 91ms/step - loss: 83.5858 - categorical_accuracy: 0.3059
Epoch 6/2000
3/3 [==============================] - 0s 94ms/step - loss: 147.0761 - categorical_accuracy: 0.3176
Epoch 7/2000
3/3 [==============================] - 0s 108ms/step - loss: 153.2458 - categorical_accuracy: 0.32940s - loss: 153.2458 - categorical_accuracy: 0.32
Epoch 8/2000
3/3 [==============================] - 0s 112ms/step - loss: 136.0985 - categorical_accuracy: 0.2824
Epoch 9/2000
3/3 [==============================

3/3 [==============================] - 0s 93ms/step - loss: 0.3304 - categorical_accuracy: 0.8471
Epoch 146/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.3522 - categorical_accuracy: 0.8471
Epoch 147/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.2803 - categorical_accuracy: 0.8824
Epoch 148/2000
3/3 [==============================] - 0s 105ms/step - loss: 0.3066 - categorical_accuracy: 0.8471
Epoch 149/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.2757 - categorical_accuracy: 0.9176
Epoch 150/2000
3/3 [==============================] - 0s 93ms/step - loss: 0.2962 - categorical_accuracy: 0.9059
Epoch 151/2000
3/3 [==============================] - 0s 91ms/step - loss: 0.3587 - categorical_accuracy: 0.8235
Epoch 152/2000
3/3 [==============================] - 0s 90ms/step - loss: 0.3705 - categorical_accuracy: 0.8000
Epoch 153/2000
3/3 [==============================] - 0s 97ms/step - loss: 0.5081 - categorical_accuracy: 0.77

3/3 [==============================] - 0s 90ms/step - loss: 0.3817 - categorical_accuracy: 0.8118
Epoch 288/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.4734 - categorical_accuracy: 0.8235
Epoch 289/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.4696 - categorical_accuracy: 0.8118
Epoch 290/2000
3/3 [==============================] - 0s 98ms/step - loss: 0.4765 - categorical_accuracy: 0.7882
Epoch 291/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.3964 - categorical_accuracy: 0.8235
Epoch 292/2000
3/3 [==============================] - 0s 96ms/step - loss: 0.4311 - categorical_accuracy: 0.8000
Epoch 293/2000
3/3 [==============================] - 0s 97ms/step - loss: 0.3894 - categorical_accuracy: 0.8353
Epoch 294/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.4965 - categorical_accuracy: 0.8000
Epoch 295/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.5584 - categorical_accuracy: 0.717

Epoch 359/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.4101 - categorical_accuracy: 0.8235
Epoch 360/2000
3/3 [==============================] - 0s 115ms/step - loss: 0.5391 - categorical_accuracy: 0.7412
Epoch 361/2000
3/3 [==============================] - 0s 107ms/step - loss: 0.4345 - categorical_accuracy: 0.7765
Epoch 362/2000
3/3 [==============================] - 0s 109ms/step - loss: 0.4022 - categorical_accuracy: 0.8353
Epoch 363/2000
3/3 [==============================] - 0s 116ms/step - loss: 0.3317 - categorical_accuracy: 0.8706
Epoch 364/2000
3/3 [==============================] - 0s 94ms/step - loss: 0.4201 - categorical_accuracy: 0.8471
Epoch 365/2000
3/3 [==============================] - 0s 103ms/step - loss: 0.3635 - categorical_accuracy: 0.8471
Epoch 366/2000
3/3 [==============================] - 0s 111ms/step - loss: 0.3892 - categorical_accuracy: 0.8235
Epoch 367/2000
3/3 [==============================] - 0s 100ms/step - loss: 0.2740 - cate

3/3 [==============================] - 0s 75ms/step - loss: 0.1653 - categorical_accuracy: 0.9529
Epoch 503/2000
3/3 [==============================] - 0s 75ms/step - loss: 0.1913 - categorical_accuracy: 0.9294
Epoch 504/2000
3/3 [==============================] - 0s 73ms/step - loss: 0.1739 - categorical_accuracy: 0.9529
Epoch 505/2000
3/3 [==============================] - 0s 79ms/step - loss: 0.1426 - categorical_accuracy: 0.9647
Epoch 506/2000
3/3 [==============================] - 0s 71ms/step - loss: 0.1397 - categorical_accuracy: 0.9529
Epoch 507/2000
3/3 [==============================] - 0s 75ms/step - loss: 0.1685 - categorical_accuracy: 0.9529
Epoch 508/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.1375 - categorical_accuracy: 0.9647
Epoch 509/2000
3/3 [==============================] - 0s 76ms/step - loss: 0.1307 - categorical_accuracy: 0.9647
Epoch 510/2000
3/3 [==============================] - 0s 75ms/step - loss: 0.1182 - categorical_accuracy: 0.988

3/3 [==============================] - 0s 88ms/step - loss: 0.2256 - categorical_accuracy: 0.8941
Epoch 646/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.1795 - categorical_accuracy: 0.9176
Epoch 647/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.1505 - categorical_accuracy: 0.9412
Epoch 648/2000
3/3 [==============================] - 0s 134ms/step - loss: 0.1916 - categorical_accuracy: 0.9176
Epoch 649/2000
3/3 [==============================] - 0s 95ms/step - loss: 0.1027 - categorical_accuracy: 0.9529
Epoch 650/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.0448 - categorical_accuracy: 1.0000
Epoch 651/2000
3/3 [==============================] - 0s 83ms/step - loss: 0.0675 - categorical_accuracy: 0.9882
Epoch 652/2000
3/3 [==============================] - 0s 99ms/step - loss: 0.0496 - categorical_accuracy: 1.0000
Epoch 653/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0488 - categorical_accuracy: 0.98

3/3 [==============================] - 0s 80ms/step - loss: 0.0382 - categorical_accuracy: 1.0000
Epoch 790/2000
3/3 [==============================] - 0s 82ms/step - loss: 0.0275 - categorical_accuracy: 1.0000
Epoch 791/2000
3/3 [==============================] - 0s 89ms/step - loss: 0.0606 - categorical_accuracy: 0.9647
Epoch 792/2000
3/3 [==============================] - 0s 80ms/step - loss: 0.0833 - categorical_accuracy: 0.9647
Epoch 793/2000
3/3 [==============================] - 0s 78ms/step - loss: 0.0407 - categorical_accuracy: 0.9882
Epoch 794/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.0325 - categorical_accuracy: 0.9882
Epoch 795/2000
3/3 [==============================] - 0s 88ms/step - loss: 0.0285 - categorical_accuracy: 0.9882 0s - loss: 0.0319 - categorical_accuracy: 0.
Epoch 796/2000
3/3 [==============================] - 0s 87ms/step - loss: 0.0245 - categorical_accuracy: 1.0000
Epoch 797/2000
3/3 [==============================] - 0s 82ms/step

KeyboardInterrupt: 

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 30, 64)            442112    
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
________________________________________________

<b> Make Predictions

In [27]:
res = model.predict(X_test)

In [43]:
actions[np.argmax(res[4])]

'iloveyou'

In [44]:
actions[np.argmax(y_test[4])]

'iloveyou'

<b> Save weights
    

In [45]:
model.save('action.h5')

<b> Evaluation using Confusion Matrix and Accuracy

In [46]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [53]:
yhat = model.predict(X_train)

In [54]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [49]:
ytrue

[1, 0, 1, 0, 2]

In [50]:
yhat

[1, 0, 1, 0, 2]

In [55]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[57,  0],
        [ 0, 28]],

       [[57,  0],
        [ 0, 28]],

       [[56,  0],
        [ 0, 29]]], dtype=int64)

In [56]:
accuracy_score(ytrue, yhat)

1.0

<b> Testing in real time

In [67]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [72]:
# 1. New detection variables
sequence = []  #collect the 30 frames to pass to prediction
sentence = []  #concatenate the history
threshold = 0.8  #render results only if they are above threshold

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
        
        
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti